## How many Bitcoin transactions are made per month?

Working with a Bitcoin dataset (Bitcoin is a popular but volatile cryptocurrency). We're going to use a **common table expression (CTE)** to find out how many Bitcoin transactions were made per month for the entire timespan of this dataset.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# import package with helper functions 
import bq_helper

# create a helper object for this dataset
bitcoin_blockchain = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                              dataset_name="bitcoin_blockchain")

Now we're going to write a query to get the number of transactions per month. One problem here is that this dataset uses timestamps rather than dates, and they're stored in this dataset as integers. We'll have to convert these into a format that BigQuery recognizes using TIMESTAMP_MILLIS(). We can do that using a CTE and then write a second part of the query against the new, temporary table we created. This has the advantage of breaking up our query into two, logical parts.

- Convert the integer to a timestamp
- Get information on the date of transactions from the timestamp

In [ ]:
query = """ WITH time AS 
            (
                SELECT TIMESTAMP_MILLIS(timestamp) AS trans_time,
                    transaction_id
                FROM `bigquery-public-data.bitcoin_blockchain.transactions`
            )
            SELECT COUNT(transaction_id) AS transactions,
                EXTRACT(MONTH FROM trans_time) AS month,
                EXTRACT(YEAR FROM trans_time) AS year
            FROM time
            GROUP BY year, month 
            ORDER BY year, month
        """

# note that max_gb_scanned is set to 21, rather than 1
transactions_per_month = bitcoin_blockchain.query_to_pandas_safe(query, max_gb_scanned=21)

Since they're returned sorted, we can just plot the raw results to show us the number of Bitcoin transactions per month over the whole timespan of this dataset.

In [ ]:
# import plotting library
import matplotlib.pyplot as plt

# plot monthly bitcoin transactions
plt.plot(transactions_per_month.transactions)
plt.title("Monthly Bitcoin Transcations")

Common table expressions let you shift a lot of your data cleaning into SQL. That's an especially good thing in the case of BigQuery because it lets you take advantage of BigQuery's parallelization, which means you'll get your results more quickly.

## Scavenger Hunt

**1. How many Bitcoin transactions were made each day in 2017?**
    
- You can use the "timestamp" column from the "transactions" table to answer this question. You can check the notebook from Day 3 for more information on timestamps.

In [ ]:
query1 = """ WITH time AS 
            (
                SELECT TIMESTAMP_MILLIS(timestamp) AS trans_time,
                    transaction_id
                FROM `bigquery-public-data.bitcoin_blockchain.transactions`
            )
            SELECT EXTRACT(DAYOFYEAR FROM trans_time) AS day,
                   COUNT(transaction_id) AS transactions
            FROM time
            WHERE EXTRACT(YEAR FROM trans_time) = 2017
            GROUP BY day 
            ORDER BY day
        """
transactions_per_day = bitcoin_blockchain.query_to_pandas_safe(query1, max_gb_scanned=21)

In [ ]:
transactions_per_day

It's better rendered as a graph...

In [ ]:
plt.plot(transactions_per_day.transactions)
plt.title("Daily Bitcoin Transcations for 2017")

**2. How many transactions are associated with each merkle root?**

- You can use the "merkle_root" and "transaction_id" columns in the "transactions" table to answer this question.

In [ ]:
query2 = """ SELECT merkle_root,
                   COUNT(transaction_id) AS transactions
            FROM `bigquery-public-data.bitcoin_blockchain.transactions`
            GROUP BY merkle_root
            ORDER BY 2 DESC
        """
transactions_per_merkle_root = bitcoin_blockchain.query_to_pandas_safe(query2, max_gb_scanned=37)

In [ ]:
transactions_per_merkle_root

That definitely needs a graph!

In [ ]:
plt.plot(transactions_per_merkle_root.transactions)
plt.title("Transactions per Merkle Root")

Use a CTE to put transactions into buckets to view summaries of the numbers

In [ ]:
query3 = """ WITH mr_trans AS (
             SELECT merkle_root,
                   COUNT(transaction_id) AS transactions
            FROM `bigquery-public-data.bitcoin_blockchain.transactions`
            GROUP BY merkle_root
            )
            , mr_grouped AS (
             SELECT 1 AS grOrder, '> 10,000' AS MerkleGroup, COUNT( merkle_root) AS GroupCount
             FROM mr_trans
             WHERE transactions >= 10000
             UNION ALL
             SELECT 2, '7,500 - 10,000', COUNT( merkle_root)
             FROM mr_trans
             WHERE transactions BETWEEN 7500 AND 9999
             UNION ALL
             SELECT 3, '5,000 - 7,500', COUNT( merkle_root)
             FROM mr_trans
             WHERE transactions BETWEEN 5000 AND 7499
             UNION ALL
             SELECT 4, '2,500 - 5,000', COUNT( merkle_root)
             FROM mr_trans
             WHERE transactions BETWEEN 2500 AND 4999
             UNION ALL
             SELECT 5, '1 - 2,500', COUNT( merkle_root)
             FROM mr_trans
             WHERE transactions < 2500             
             )
             SELECT MerkleGroup, GroupCount 
             FROM mr_grouped
             ORDER BY grOrder
        """
transactions_per_merkle_root_group = bitcoin_blockchain.query_to_pandas_safe(query3, max_gb_scanned=37)


In [ ]:
print(transactions_per_merkle_root_group)